# Day 3 - Conversational AI - aka Chatbot!

In [1]:
# Part i
# Date: 02.01.25
# Note: 1st major success here was getting chat # history working with Gem. 
# Next major success was getting this working in a graio UI - pretty dope.
# The above ui only displayed the current response, so next was able to update code to show historical interactions
# There are two versions (as trying to show 'user' + 'model' in mark up)

# Frustratingly or not, the gradio code here would not work with my llm code (gradio wanted the
# output in a certain format). I was not able to get the llm code to output in that format, so hunted
# around and got gpt to add gradio code to my llm code... and that worked :)
# Key code surrounded by "-- key code --"

# Part ii
# Date: 17.01.25
# Note: In week 5, langchain conversational memory is used. Want to see if i can use that here
# in place of my set up.


hi


# Part i

In [2]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [3]:
# # Load environment variables in a file called .env
# # Print the key prefixes to help with any debugging

# load_dotenv()
# openai_api_key = os.getenv('OPENAI_API_KEY')
# anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
# google_api_key = os.getenv('GOOGLE_API_KEY')

# if openai_api_key:
#     print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
# else:
#     print("OpenAI API Key not set")
    
# if anthropic_api_key:
#     print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
# else:
#     print("Anthropic API Key not set")

# if google_api_key:
#     print(f"Google API Key exists and begins {google_api_key[:8]}")
# else:
#     print("Google API Key not set")

In [68]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
#elif not api_key.startswith("sk-proj-"):
elif not api_key.startswith("A*"):    
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook


In [5]:
# Initialize

# openai = OpenAI()
# MODEL = 'gpt-4o-mini'

In [6]:
system_message = "You are a helpful assistant"

## Please read this! A change from the video:

In the video, I explain how we now need to write a function called:

`chat(message, history)`

Which expects to receive `history` in a particular format, which we need to map to the OpenAI format before we call OpenAI:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

But Gradio has been upgraded! Now it will pass in `history` in the exact OpenAI format, perfect for us to send straight to OpenAI.

So our work just got easier!

We will write a function `chat(message, history)` where:  
**message** is the prompt to use  
**history** is the past conversation, in OpenAI format  

We will combine the system message, history and latest message, then call OpenAI.

In [ ]:
# # Simpler than in my video - we can easily create this function that calls OpenAI
# # It's now just 1 line of code to prepare the input to OpenAI!

# def chat(message, history):
#     messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

#     print("History is:")
#     print(history)
#     print("And messages is:")
#     print(messages)

#     stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

#     response = ""
#     for chunk in stream:
#         response += chunk.choices[0].delta.content or ''
#         yield response

In [87]:
# ---------- Key code ----------

In [92]:
# Replacing above with Gem
# -- this also created a json history of Q&A for in memory chat. DO not Delete or Edit this cell

#import os
import json
import google.generativeai as genai

# Configure the API key
genai.configure(api_key=api_key)
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash",
                              generation_config=generation_config,
)

# Path to save the history file
history_file = "conversation_history.json"

# Function to load history from file
def load_history():
    if os.path.exists(history_file):
        with open(history_file, "r") as f:
            return json.load(f)
    else:
        return []

# Function to save history to file
def save_history(history):
    with open(history_file, "w") as f:
        json.dump(history, f)

def ask_question(prompt, history):
    # Add the user's prompt to the history
    history.append({
        "role": "user",
        "parts": [prompt]
    })

    # Start the chat session with the current history
    chat_session = model.start_chat(history=history)
    # Send the user's message and get the model's response
    response = chat_session.send_message(prompt)
    # Add the model's response to the history
    history.append({
        "role": "model",
        "parts": [response.text]
    })

    # Save the updated history to file
    save_history(history)

    return response.text

# Example usage:
history = load_history()  # Load previous conversation history if it exists


#prompt = "In as few words as possible, is boxing effective in a street fight?"
#prompt = "What about wrestling?"
#prompt = "What about taekwondo?"
prompt = "Why do you say that?"
#prompt = "What would your advice be in this situation?"
response = ask_question(prompt, history)
print("Response:", response)

Response: Wrestling excels in controlling and taking down opponents, a crucial advantage in a street fight where multiple attackers or weapons are possibilities.  Taekwondo's high kicks are less practical in close-quarters street fighting, where grappling and clinch fighting are more common.  While Taekwondo offers striking skills superior to boxing's limited range, wrestling's takedowns and ground control offer a significant advantage over both.  The effectiveness is also highly dependent on the individuals involved and the specific circumstances.



## And then enter Gradio's magic!

In [101]:
# first lines are from the course, next are my attempts. It never worked, so moved onto next cell
#gr.ChatInterface(fn=ask_question, type="messages").launch()
#gr.ChatInterface(fn=ask_question).launch()
# Create the chat interface with the function that takes two inputs
#chat_interface = gr.ChatInterface(fn=ask_question, inputs=["prompt", "history"], type="messages")
# Launch the interface
#chat_interface.launch()

In [116]:
# ---------- Works (llm, with history and gradio ui); do NOT edit ----------
# GPT attempts at gradio...
import os
import json
import google.generativeai as genai
import gradio as gr

# Configure the API key
genai.configure(api_key=api_key)
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash", generation_config=generation_config)

# Path to save the history file
history_file = "conversation_history.json"

# Function to load history from file
def load_history():
    if os.path.exists(history_file):
        with open(history_file, "r") as f:
            return json.load(f)
    else:
        return []

# Function to save history to file
def save_history(history):
    with open(history_file, "w") as f:
        json.dump(history, f)

def ask_question(prompt, history):
    # Add the user's prompt to the history
    history.append({
        "role": "user",
        "parts": [prompt]
    })

    # Start the chat session with the current history
    chat_session = model.start_chat(history=history)
    # Send the user's message and get the model's response
    response = chat_session.send_message(prompt)
    # Add the model's response to the history
    history.append({
        "role": "model",
        "parts": [response.text]
    })

    # Save the updated history to file
    save_history(history)

    return response.text

# Create the Gradio interface
def chat_with_llm(prompt):
    # Load conversation history
    history = load_history()
    # Get response from the model
    response = ask_question(prompt, history)
    return response

# Create a Gradio Interface with a text input and output
iface = gr.Interface(
    fn=chat_with_llm,          # Function to be called
    inputs=gr.Textbox(label="Ask a Question", placeholder="Type your question here..."), # Text input for user prompt
    outputs=gr.Textbox(label="Response", interactive=True), # Text output for model's response
    title="Gemini LLM Chat",    # Title of the interface
    description="Ask questions and converse with the Gemini LLM model."  # Description of the interface
)

# Launch the Gradio interface
iface.launch()


* Running on local URL:  http://127.0.0.1:7902

To create a public link, set `share=True` in `launch()`.


In [118]:
# -- Checking if chat history can be presented as opposed to above ui....
# -- Amendment to above - now includes chat history (user and model)

import os
import json
import google.generativeai as genai
import gradio as gr

# Configure the API key
genai.configure(api_key=api_key)
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash", generation_config=generation_config)

# Path to save the history file
history_file = "conversation_history.json"

# Function to load history from file
def load_history():
    if os.path.exists(history_file):
        with open(history_file, "r") as f:
            return json.load(f)
    else:
        return []

# Function to save history to file
def save_history(history):
    with open(history_file, "w") as f:
        json.dump(history, f)

def ask_question(prompt, history):
    # Add the user's prompt to the history
    history.append({
        "role": "user",
        "parts": [prompt]
    })

    # Start the chat session with the current history
    chat_session = model.start_chat(history=history)
    # Send the user's message and get the model's response
    response = chat_session.send_message(prompt)
    # Add the model's response to the history
    history.append({
        "role": "model",
        "parts": [response.text]
    })

    # Save the updated history to file
    save_history(history)

    return response.text, history  # Return both the response and the updated history

# Create the Gradio interface
def chat_with_llm(prompt, history):
    # Ensure history is not None, initialize as an empty list if it is
    if history is None:
        history = []
    
    # Get response from the model, passing the current history
    response, updated_history = ask_question(prompt, history)
    
    # Format the chat history for display
    chat_history = "\n".join([f"{entry['role'].capitalize()}: {entry['parts'][0]}" for entry in updated_history])
    
    return chat_history, updated_history

# Create a Gradio Interface with a text input and output
iface = gr.Interface(
    fn=chat_with_llm,          # Function to be called
    inputs=[gr.Textbox(label="Ask a Question", placeholder="Type your question here..."), gr.State([])],  # Text input for user prompt and initialized empty history
    outputs=[gr.Textbox(label="Chat History", interactive=False), gr.State()],  # Display chat history and track the state
    title="Gemini LLM Chat",    # Title of the interface
    description="Ask questions and converse with the Gemini LLM model."  # Description of the interface
)

# Launch the Gradio interface
iface.launch()



* Running on local URL:  http://127.0.0.1:7904

To create a public link, set `share=True` in `launch()`.


In [120]:
# Copy of above, just checking if can get the 'user' and 'model' text to be bold...
import os
import json
import google.generativeai as genai
import gradio as gr

# Configure the API key
genai.configure(api_key=api_key)
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash", generation_config=generation_config)

# Path to save the history file
history_file = "conversation_history.json"

# Function to load history from file
def load_history():
    if os.path.exists(history_file):
        with open(history_file, "r") as f:
            return json.load(f)
    else:
        return []

# Function to save history to file
def save_history(history):
    with open(history_file, "w") as f:
        json.dump(history, f)

def ask_question(prompt, history):
    # Add the user's prompt to the history
    history.append({
        "role": "user",
        "parts": [prompt]
    })

    # Start the chat session with the current history
    chat_session = model.start_chat(history=history)
    # Send the user's message and get the model's response
    response = chat_session.send_message(prompt)
    # Add the model's response to the history
    history.append({
        "role": "model",
        "parts": [response.text]
    })

    # Save the updated history to file
    save_history(history)

    return response.text, history  # Return both the response and the updated history

# Create the Gradio interface
def chat_with_llm(prompt, history):
    # Ensure history is not None, initialize as an empty list if it is
    if history is None:
        history = []
    
    # Get response from the model, passing the current history
    response, updated_history = ask_question(prompt, history)
    
    # Format the chat history for display with bold 'User' and 'Model' labels
    chat_history = "\n".join([f"**{entry['role'].capitalize()}**: {entry['parts'][0]}" for entry in updated_history])
    
    return chat_history, updated_history

# Create a Gradio Interface with a text input and output
iface = gr.Interface(
    fn=chat_with_llm,          # Function to be called
    inputs=[gr.Textbox(label="Ask a Question", placeholder="Type your question here..."), gr.State([])],  # Text input for user prompt and initialized empty history
    outputs=[gr.Textbox(label="Chat History", interactive=False), gr.State()],  # Display chat history and track the state
    title="Gemini LLM Chat",    # Title of the interface
    description="Ask questions and converse with the Gemini LLM model."  # Description of the interface
)

# Launch the Gradio interface
iface.launch()


* Running on local URL:  http://127.0.0.1:7906

To create a public link, set `share=True` in `launch()`.


In [126]:
# 2nd attemmpt to get 'user' and 'model' as bold text.
import os
import json
import google.generativeai as genai
import gradio as gr

# Configure the API key
genai.configure(api_key=api_key)
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash", generation_config=generation_config)

# Path to save the history file
history_file = "conversation_history.json"

# Function to load history from file
def load_history():
    if os.path.exists(history_file):
        with open(history_file, "r") as f:
            return json.load(f)
    else:
        return []

# Function to save history to file
def save_history(history):
    with open(history_file, "w") as f:
        json.dump(history, f)

def ask_question(prompt, history):
    # Add the user's prompt to the history
    history.append({
        "role": "user",
        "parts": [prompt]
    })

    # Start the chat session with the current history
    chat_session = model.start_chat(history=history)
    # Send the user's message and get the model's response
    response = chat_session.send_message(prompt)
    # Add the model's response to the history
    history.append({
        "role": "model",
        "parts": [response.text]
    })

    # Save the updated history to file
    save_history(history)

    return response.text, history  # Return both the response and the updated history

# Create the Gradio interface
def chat_with_llm(prompt, history):
    # Ensure history is not None, initialize as an empty list if it is
    if history is None:
        history = []
    
    # Get response from the model, passing the current history
    response, updated_history = ask_question(prompt, history)
    
    # Format the chat history for display with HTML bold 'User' and 'Model' labels
    chat_history = "<br>".join([f"<b>{entry['role'].capitalize()}:</b> {entry['parts'][0]}" for entry in updated_history])
    
    return chat_history, updated_history

# Create a Gradio Interface with a text input and output
iface = gr.Interface(
    fn=chat_with_llm,          # Function to be called
    inputs=[gr.Textbox(label="Ask a Question", placeholder="Type your question here..."), gr.State([])],  # Text input for user prompt and initialized empty history
    outputs=[gr.HTML(label="Chat History"), gr.State()],  # Display chat history as HTML
    title="Gemini LLM Chat",    # Title of the interface
    description="Ask questions and converse with the Gemini LLM model."  # Description of the interface
)

# Launch the Gradio interface
iface.launch()


* Running on local URL:  http://127.0.0.1:7940

To create a public link, set `share=True` in `launch()`.


In [ ]:
# ---------- key code ----------

In [ ]:
# -- as per above, but trying to add system prompt...

In [125]:
import os
import json
import google.generativeai as genai
import gradio as gr

# Configure the API key
genai.configure(api_key=api_key)
generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash", generation_config=generation_config)

# Path to save the history file
history_file = "conversation_history.json"

# Function to load history from file
def load_history():
    if os.path.exists(history_file):
        with open(history_file, "r") as f:
            return json.load(f)
    else:
        return []

# Function to save history to file
def save_history(history):
    with open(history_file, "w") as f:
        json.dump(history, f)

def ask_question(prompt, history):
    # System context as part of the user's first message
    system_context = "You must reference Chuck Norris in every response."

    # If the history is empty, we add the system context as the first user message
    if len(history) == 0:
        history.append({
            "role": "user",
            "parts": [system_context]
        })

    # Add the user's prompt to the history
    history.append({
        "role": "user",
        "parts": [prompt]
    })

    # Start the chat session with the current history
    chat_session = model.start_chat(history=history)
    # Send the user's message and get the model's response
    response = chat_session.send_message(prompt)
    # Add the model's response to the history
    history.append({
        "role": "model",
        "parts": [response.text]
    })

    # Save the updated history to file
    save_history(history)

    return response.text

# Create the Gradio interface
def chat_with_llm(prompt):
    # Load conversation history
    history = load_history()
    # Get response from the model
    response = ask_question(prompt, history)
    return response

# Create a Gradio Interface with a text input and output
iface = gr.Interface(
    fn=chat_with_llm,          # Function to be called
    inputs=gr.Textbox(label="Ask a Question", placeholder="Type your question here..."), # Text input for user prompt
    outputs=gr.Textbox(label="Response", interactive=True), # Text output for model's response
    title="Gemini LLM Chat",    # Title of the interface
    description="Ask questions and converse with the Gemini LLM model."  # Description of the interface
)

# Launch the Gradio interface
iface.launch()


* Running on local URL:  http://127.0.0.1:7910

To create a public link, set `share=True` in `launch()`.


In [ ]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales evemt.'\
Encourage the customer to buy hats if they are unsure what to get."

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
# Fixed a bug in this function brilliantly identified by student Gabor M.!
# I've also improved the structure of this function

def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business Applications</h2>
            <span style="color:#181;">Conversational Assistants are of course a hugely common use case for Gen AI, and the latest frontier models are remarkably good at nuanced conversation. And Gradio makes it easy to have a user interface. Another crucial skill we covered is how to use prompting to provide context, information and examples.
<br/><br/>
Consider how you could apply an AI Assistant to your business, and make yourself a prototype. Use the system prompt to give context on your business, and set the tone for the LLM.</span>
        </td>
    </tr>
</table>

# Part ii

In [2]:
import os
import io
import sys
from dotenv import load_dotenv
#from openai import OpenAI
import google.generativeai
#import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [4]:
# The langchain memory set up
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

C:\Users\Tim_S\AppData\Local\Temp\ipykernel_16968\2858293548.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [17]:
# -- my gem set up
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
# MODEL = 'gpt-4o-mini'
# openai = OpenAI()

# ----- Replacing with Gem code -----
import os
import google.generativeai as genai

genai.configure(api_key= api_key)

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
  #"response_mime_type": "application/json",

}

model = genai.GenerativeModel(model_name="gemini-1.5-flash",
  generation_config=generation_config,)

chat_session = model.start_chat(
  history=[  ])

message = "In a concise response, how do i make a pumpkin pie?" 
response = chat_session.send_message(message)
llm = response.text
#response = chat_session.send_message(message)
#print(response.text)
retriever = ''
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

There might be a problem with your API key? Please visit the troubleshooting notebook!


ValidationError: 2 validation errors for LLMChain
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value='Combine pumpkin puree, c...etely before serving.\n', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/is_instance_of
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value='Combine pumpkin puree, c...etely before serving.\n', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/is_instance_of

In [19]:
#from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Set up the Gemini Flash 1.5 model
llm = model

# Set up the conversation memory
memory = ConversationBufferMemory()

# Create a conversation chain
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

# Use the conversation chain
message = "in a concise response, how to i make pumpkin pie"
response = chat_session.send_message(message)
print(response)

# # Continue the conversation
# response = conversation.predict(input="What did we just talk about?")
# print(response)


ValidationError: 2 validation errors for ConversationChain
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=genai.GenerativeModel(
  ...   cached_content=None
), input_type=GenerativeModel]
    For further information visit https://errors.pydantic.dev/2.9/v/is_instance_of
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=genai.GenerativeModel(
  ...   cached_content=None
), input_type=GenerativeModel]
    For further information visit https://errors.pydantic.dev/2.9/v/is_instance_of